In [1]:
import pyspark

In [2]:
pyspark.__file__

'/home/maxpr/spark/spark-3.0.3-bin-hadoop3.2/python/pyspark/__init__.py'

In [7]:
!mkdir data
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv -P data

mkdir: cannot create directory ‘data’: File exists
--2022-02-27 09:36:54--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.162.72
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.162.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘data/taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-27 09:36:54 (59.8 MB/s) - ‘data/taxi+_zone_lookup.csv.1’ saved [12322/12322]



In [33]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df_zones = spark.read \
    .option("header", "true") \
    .csv('data/taxi+_zone_lookup.csv')

df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

### http://localhost:4040/jobs/ contains the spark job 

# Homework

## Q1

In [ ]:
spark.version

In [14]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv -P data

--2022-02-26 06:06:08--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.139.169
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.139.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘data/fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  70.6MB/s    in 8.6s    

2022-02-26 06:06:17 (81.0 MB/s) - ‘data/fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [41]:
!du -sh data/fhvhv_tripdata_2021-02.csv

700M	data/fhvhv_tripdata_2021-02.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema",True) \
    .csv('data/fhvhv_tripdata_2021-02.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,IntegerType,true)))

## Question 2 
Repartition it to 24 partitions and save it to parquet.

What's the size of the folder with results (in MB)?

In [46]:
!rm -rf data/repartition24/

In [47]:
df = df.repartition(24)
df.write.parquet('data/repartition24/2021/02')

In [48]:
!du -sh data/repartition24

210M	data/repartition24


## To convert 
- pickup_datetime -> datetime
- dropoff_datetime -> datetime

In [6]:
from pyspark.sql import functions

In [7]:
df = df.withColumn("pickup_date", functions.to_date(df.pickup_datetime))

In [8]:
df = df.withColumn("dropoff_date", functions.to_date(df.dropoff_datetime))

In [9]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,IntegerType,true),StructField(pickup_date,DateType,true),StructField(dropoff_date,DateType,true)))

## Question 3. Count records

How many taxi trips were there on February 15? (Consider only trips that started on February 15)

In [10]:
df.filter(df.pickup_date == '2021-02-15').count()

367170

## Question 4. Longest trip for each day
Now calculate the duration for each trip.

Trip starting on which day was the longest?

In [11]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null| 2021-02-01|  2021-02-01|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null| 2021-02-01|  2021-02-01|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null| 2021-02-01|  2021-02-01|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null| 2021-02-01|  2021-02-01|
|           HV0003| 

In [15]:
from pyspark.sql.functions import *

df = df.withColumn('from_timestamp',to_timestamp(col('pickup_datetime'))) \
     .withColumn('end_timestamp', to_timestamp(col('dropoff_datetime'))) \
     .withColumn('trip_duration_in_second',unix_timestamp("end_timestamp") - unix_timestamp('from_timestamp'))

In [16]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------------------+-------------------+-----------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|     from_timestamp|      end_timestamp|trip_duration_in_second|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------------------+-------------------+-----------------------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null| 2021-02-01|  2021-02-01|2021-02-01 00:10:40|2021-02-01 00:21:09|                    629|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null| 2021-02-01|  2021-02-01|2021-02-01 00:27:23|2021-02-01 00:44:01|     

In [28]:
df.groupBy('pickup_date') \
  .agg(max("trip_duration_in_second").alias("max_trip")) \
  .sort(desc("max_trip")) \
  .show()

+-----------+--------+
|pickup_date|max_trip|
+-----------+--------+
| 2021-02-11|   75540|
| 2021-02-17|   57221|
| 2021-02-20|   44039|
| 2021-02-03|   40653|
| 2021-02-19|   37577|
| 2021-02-25|   35010|
| 2021-02-18|   34612|
| 2021-02-10|   34169|
| 2021-02-21|   32223|
| 2021-02-09|   32087|
| 2021-02-06|   31447|
| 2021-02-02|   30913|
| 2021-02-05|   30511|
| 2021-02-12|   30148|
| 2021-02-08|   30106|
| 2021-02-14|   29777|
| 2021-02-22|   28278|
| 2021-02-27|   27170|
| 2021-02-15|   25874|
| 2021-02-04|   25592|
+-----------+--------+
only showing top 20 rows



## Question 5. Most frequent dispatching_base_num
Now find the most frequently occurring dispatching_base_num in this dataset.

How many stages this spark job has?

In [32]:
df.groupBy('dispatching_base_num') \
  .count().alias("count") \
  .sort(desc("count")) \
  .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



## Question 6. Most common locations pair

Find the most common pickup-dropoff pair.

For example:

"Jamaica Bay / Clinton East"

Enter two zone names separated by a slash

If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

In [41]:

df_joined = df.join(df_zones.select("LocationID", "Zone"), df.PULocationID == df_zones.LocationID, "left") \
        .drop("LocationID") \
        .withColumnRenamed("Zone", "PULocation") \
    .join(df_zones.select("LocationID", "Zone"), df.DOLocationID == df_zones.LocationID, "left") \
        .drop("LocationID") \
        .withColumnRenamed("Zone", "DOLocation")

In [50]:
df_joined = df_joined.withColumn("LocationPair", functions.concat_ws(" / ", df_joined["PULocation"],df_joined["DOLocation"]))

In [51]:
df_joined.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------------------+-------------------+-----------------------+--------------------+--------------------+--------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|     from_timestamp|      end_timestamp|trip_duration_in_second|          PULocation|          DOLocation|        LocationPair|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------------------+-------------------+-----------------------+--------------------+--------------------+--------------------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null| 2021-02-01|  2021-02-01|2021-02-01 00:10:40|2021-02-01 00:21:09|                    

In [52]:
df_joined.groupBy("LocationPair").count().alias("count").sort(desc("count")).show()

+--------------------+-----+
|        LocationPair|count|
+--------------------+-----+
|East New York / E...|45041|
|Borough Park / Bo...|37329|
| Canarsie / Canarsie|28026|
|Crown Heights Nor...|25976|
|Bay Ridge / Bay R...|17934|
|Jackson Heights /...|14688|
|   Astoria / Astoria|14688|
|Central Harlem No...|14481|
|Bushwick South / ...|14424|
|Flatbush/Ditmas P...|13976|
|South Ozone Park ...|13716|
|Brownsville / Bro...|12829|
|    JFK Airport / NA|12542|
|Prospect-Lefferts...|11814|
|Forest Hills / Fo...|11548|
|Bushwick North / ...|11491|
|Bushwick South / ...|11487|
|Crown Heights Nor...|11462|
|Crown Heights Nor...|11342|
|Prospect-Lefferts...|11308|
+--------------------+-----+
only showing top 20 rows

